# Sampe Resume Generator

履歴書の自己PR文の例を自動生成します。
LangChain の勉強のために作成したサンプルプログラムです。

## 実行手順

1. OpenAIのAPIキーを発行してください。以下を参照：
  * [OpenAI API キーを取得する方法](https://laboratory.kazuuu.net/how-to-get-an-openai-api-key/)
  * [【エンジニア向け】OpenAIのAPI連携方法【環境構築 + GASによるGoogle Documentへの組み込み】](https://youtu.be/frpsKLNW1q4)
1. 以下の入力項目を全て記入してください。
1. [全てのセルを実行] を行ってください。


In [ ]:
#@title 入力項目
#@markdown ### ●OpenAIのAPIキーを入力してください。
open_api_key = '' #@param {type:"string"}

#@markdown ### ●応募先の企業や組織の説明を入力してください。業種、規模、業界での位置づけ、得意分野など、できるだけ詳しく記入してください。
organization = '\u8A2D\u7ACB\uFF13\u5E74\u76EE\u306E\u30D9\u30F3\u30C1\u30E3\u30FC\u4F01\u696D\u3067\u3059\u3002\u5F93\u696D\u54E1\u6570\u306F4\u540D\u3067\u3059\u3002AI\u6280\u8853\u3092\u9069\u7528\u3057\u305F\u696D\u52D9\u30B7\u30B9\u30C6\u30E0\u306E\u5B9F\u73FE\u306B\u5411\u3051\u305FPoC\u304A\u3088\u3073\u30B7\u30B9\u30C6\u30E0\u958B\u767A\u3092\u884C\u3063\u3066\u3044\u307E\u3059\u3002' #@param {type:"string"}

#@markdown ### ●あなたの長所を入力してください。
strengths = '\u81EA\u7136\u8A00\u8A9E\u51E6\u7406\u5206\u91CE\u306EAI\u6280\u8853\u3092\u4FDD\u6709\u3057\u3066\u3044\u308B\u3053\u3068\u3001\u304A\u3088\u3073\u3001\u672C\u756A\u306EAI\u30B7\u30B9\u30C6\u30E0\u306E\u30C7\u30EA\u30D0\u30EA\u7D4C\u9A13\u3092\u4FDD\u6709\u3057\u3066\u3044\u308B\u3053\u3068\u3067\u3059\u3002' #@param {type:"string"}

In [ ]:
#@title
!pip3 install langchain openai

In [ ]:
#@title
import os
os.environ["OPENAI_API_KEY"] = open_api_key

In [ ]:
#@title
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

In [ ]:
#@title
# ----- 質問者
system_template = SystemMessagePromptTemplate.from_template("""
    あなたはビジネスコンサルタントです。ある組織[X]のSWOT分析を行いました。これからその分析結果[Y]を報告するところです。
    [X]は次のような組織です： {organization}
    [Y]は、[X]の強み（Strength）、弱み（Weakness）、機会（Opportunity）、脅威（Threat）の分析結果です。それぞれ箇条書きで示されています。
    """)
human_template = HumanMessagePromptTemplate.from_template("""
    質問者：分析結果[Y]を示してください。
    """)
message_list = ChatPromptTemplate.from_messages(
    [system_template, human_template]
    ).format_prompt(organization=organization).to_messages()
print(message_list[0].content, '\n')
print(message_list[1].content, '\n')

# ----- AI
message_list += [chat(message_list)]
print('AI: ', message_list[-1].content, '\n')

# ----- 質問者
human_template = HumanMessagePromptTemplate.from_template("""
    質問者：分析結果[Y]にもとづいて、組織[X]が直面すると予想される課題[Z]を、箇条書きで示してください。
    """)
message_list += ChatPromptTemplate.from_messages(
    [human_template]
    ).format_prompt().to_messages()
print(message_list[-1].content, '\n')

# ----- AI
message_list += [chat(message_list)]
print('AI: ', message_list[-1].content, '\n')

# ----- 質問者
system_template = SystemMessagePromptTemplate.from_template("""
    今から、あなたは、就職コンサルタントです。
    """)
human_template = HumanMessagePromptTemplate.from_template("""
    質問者： 私の目的は、組織[X]の就職面接に応募して、合格することです。
    私の長所は次の通りです： {strengths}
    質問： 組織[X]の就職面接に応募するための履歴書に記載する自己アピール文の例を作成してください。自己アピール文では、組織[X]の課題[Z]の解決に、私が貢献できることをアピールしてください。
    ただし、[Z]に含まれる課題の内、私が貢献することが難しい課題については、言及する必要はありません。
    """)
message_list += ChatPromptTemplate.from_messages(
    [system_template, human_template]
    ).format_prompt(strengths=strengths).to_messages()
print(message_list[-2].content, '\n')
print(message_list[-1].content, '\n')

# ----- AI
message_list += [chat(message_list)]
print('AI: ', message_list[-1].content, '\n')



    あなたはビジネスコンサルタントです。ある組織[X]のSWOT分析を行いました。これからその分析結果[Y]を報告するところです。
    [X]は次のような組織です： 設立３年目のベンチャー企業です。従業員数は4名です。AI技術を適用した業務システムの実現に向けたPoCおよびシステム開発を行っています。
    [Y]は、[X]の強み（Strength）、弱み（Weakness）、機会（Opportunity）、脅威（Threat）の分析結果です。それぞれ箇条書きで示されています。
     


    質問者：分析結果[Y]を示してください。
     

AI:  分析結果[Y]は以下の通りです：

Strength:
- 技術力：[X]はAI技術を適用した業務システムのPoCおよびシステム開発を行っており、従業員が高い技術力を持っています。
- 小規模な組織：ベンチャー企業であるため、意思決定が迅速に行われ、柔軟な対応が可能です。

Weakness:
- 人材不足：従業員数がわずか4名であるため、人材不足が課題となっています。迅速な拡大が必要です。
- 経験不足：設立３年目のため、経験不足があります。市場への理解や顧客のニーズを把握する能力を向上させる必要があります。

Opportunity:
- 市場の成長：AI技術の需要が急速に成長しており、[X]のような専門知識を持った企業に需要があります。
- パートナーシップの可能性：他の企業との協業や提携により、新たなビジネスチャンスを得ることができます。

Threat:
- 競合他社の存在：AI技術を活用した業務システムの開発は競争が激しく、他の企業との競合が予想されます。
- 技術の進歩：AI技術は急速に進化しており、[X]が最新の技術に追いつくためには努力が必要です。

以上が、組織[X]のSWOT分析の結果[Y]です。 


    質問者：分析結果[Y]にもとづいて、組織[X]が直面すると予想される課題[Z]を、箇条書きで示してください。
     

AI:  組織[X]が直面すると予想される課題[Z]は以下の通りです：

- 人材不足による業務の遅れや品質の低下
- 経験不足による市場競争力の低下
- 競合他社との競争による市場シェアの低下
- 技術の進歩に追いつけず、顧客のニーズに対